# Day 23
https://adventofcode.com/2020/day/23

In [1]:
import aocd
data = aocd.get_data(year=2020, day=23)

In [2]:
from math import prod

In [3]:
class Cup():
    
    def __init__(self, identifier):
        self.identifier = identifier
        self.left = None
        self.right = None
    
    def __repr__(self):        
        return 'Cup(identifier={}, left={}, right={})'.format(
            repr(self.identifier),
            repr(None if self.left is None else self.left.identifier),
            repr(None if self.right is None else self.right.identifier),
        )
    
    def insert(self, new_right_neighbour):
        new_right_neighbour.left = self
        if self.left is None and self.right is None:
            new_right_neighbour.right = self
            self.left = new_right_neighbour
        else:
            new_right_neighbour.right = self.right
            self.right.left = new_right_neighbour
        self.right = new_right_neighbour
    
    def remove(self):
        self.left.right = self.right
        self.right.left = self.left
        self.left = None
        self.right = None
        return self

In [4]:
class Game():
    
    def __init__(self, cups, current):
        self.cups = cups
        self.current = current
    
    def __repr__(self):
        return 'Game(cups=[{} cups], current={})'.format(
            len(self.cups),
            self.current.identifier,
        )
    
    @classmethod
    def from_puzzle_input(cls, text, million_cups=False):
        cup_order = tuple(int(digit) for digit in text)
        if million_cups:
            cup_order += tuple(range(max(cup_order)+1, 1_000_001))
        
        cups = {}
        first = current = Cup(cup_order[0])
        cups[first.identifier] = first
        
        for cup_id in cup_order[1:]:
            new_cup = Cup(cup_id)
            current.insert(new_cup)
            cups[cup_id] = new_cup
            current = new_cup

        return cls(cups, first)
    
    @property
    def cup_order(self):
        order = []
        node = self.cups[1]
        for x in range(len(self.cups)-1):
            node = node.right
            order.append(node.identifier)
        return order
    
    def play(self, moves):
        for move in range(moves):
            removed = [self.current.right, self.current.right.right, self.current.right.right.right]
            for removal in removed:
                removal.remove()
            removed_ids = {removal.identifier for removal in removed}
            
            destination_id = self.current.identifier - 1
            while destination_id in removed_ids or destination_id == 0:
                destination_id -= 1
                if destination_id <= 0:
                    destination_id = max(cup_id for cup_id in self.cups)
            destination = self.cups[destination_id]
            
            for insertion in removed:
                destination.insert(insertion)
                destination = insertion
            self.current = self.current.right
        return self

In [5]:
order = Game.from_puzzle_input(data).play(100).cup_order
p1 = ''.join(str(cup_id) for cup_id in order)
print('Part 1: {}'.format(p1))

Part 1: 43896725


In [6]:
bigger_game = Game.from_puzzle_input(data, million_cups=True)
bigger_game.play(10_000_000)
interesting_cups = (bigger_game.cups[1].right, bigger_game.cups[1].right.right)
p2 = prod(cup.identifier for cup in interesting_cups)
print('Part 2: {}'.format(p2))

Part 2: 2911418906
